In [100]:
import os
import pandas as pd
import plotly.express as px
from google.cloud import bigquery

pd.set_option('max_colwidth', 70)

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/fabiomendes/Downloads/balancer-314012-de7efb77a81d.json"

In [4]:
FROM_BLOCK_TIMESTAMP = "'2020-11-02 00:00:00'"
TO_BLOCK_TIMESTAMP = "'2020-11-20 23:59:59'" 

MY_TIMEOUT = 300

client = bigquery.Client()

## Uniswap Events

In [24]:
event_hashes = {
    '0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822': 'Swap',
    '0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1': 'Sync',
    '0xdccd412f0b1252819cb1fd330b93224ca42612892bb3f4f789976e6d81936496': 'Burn',
    '0x4c209b5fc8ad50758f13e2e1088ba56a560dff690a1c6fef26394f4c03821c4f': 'Mint',
    '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef': 'Transfer',
    '0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925': 'Approval' 
}

contracts = {
    '0xa478c2975ab1ea89e8196811f51a7b7ade33eb11'.lower(): 'DAI'
}

In [109]:
query = """
SELECT 
  events.transaction_hash AS transactionHash,
  events.transaction_index,
  txns.from_address AS transaction_sender, 
  txns.receipt_gas_used AS gas_used,
  txns.gas_price,
  events.address,
  events.data,
  events.topics,
  events.block_timestamp,
  events.block_number AS blockNumber_dec
FROM
  `bigquery-public-data.crypto_ethereum.logs` AS events
INNER JOIN
  `bigquery-public-data.crypto_ethereum.transactions` AS txns
ON
  events.transaction_hash = txns.hash
WHERE TRUE
  AND events.block_timestamp >= {from_block_ts} 
  AND events.block_timestamp <= {to_block_ts}
  AND txns.block_timestamp >= {from_block_ts} --might seem redundant, but because of partitioning this reduces cost
  AND txns.block_timestamp <= {to_block_ts} --might seem redundant, but because of partitioning this reduces cost
  AND events.address IN ({contract_list})
""".format(
    from_block_ts=FROM_BLOCK_TIMESTAMP, 
    to_block_ts=TO_BLOCK_TIMESTAMP, 
    contract_list=(','.join(["'{}'".format(k) for k in list(contracts.keys())])))

query_job = client.query(query)
iterator = query_job.result(timeout=MY_TIMEOUT)
rows = list(iterator)
uniswap = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
uniswap['contract'] = uniswap['address'].apply(lambda x: contracts[x])
uniswap['event'] = (uniswap.topics.apply(lambda x: x[0]).map(event_hashes))
uniswap = uniswap.sort_values(['blockNumber_dec','transaction_index']).reset_index(drop=True)
uniswap.head(10)

,transactionHash,transaction_index,transaction_sender,gas_used,gas_price,address,data,topics,block_timestamp,blockNumber_dec,contract,event
0,0xe3a7b8ff1710c12eb4318e919d354ceb83b29aec97e619653ec71a56213f6fd2,42,0x91b41af00272757574c98676c58165edf2990ea5,114126,100500000001,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x00000000000000000000000000000000000000000098cbd0a4aec4c7e43e946b...,[0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1],2020-11-02 00:00:58+00:00,11174331,DAI,Sync
1,0xe3a7b8ff1710c12eb4318e919d354ceb83b29aec97e619653ec71a56213f6fd2,42,0x91b41af00272757574c98676c58165edf2990ea5,114126,100500000001,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x00000000000000000000000000000000000000000000022032443dbf8d9c0a7e...,[0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d82...,2020-11-02 00:00:58+00:00,11174331,DAI,Swap
2,0xc8d0d9abfcc80cbd5973c5c378d1160a85ff1e591496a1d0f3b5d4d321279e24,43,0x125bf69c61af3c60ca1c0df5bbbce503cff7b1ae,109247,100500000000,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x00000000000000000000000000000000000000000098d0a2447723f9fbbe946b...,[0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1],2020-11-02 00:00:58+00:00,11174331,DAI,Sync
3,0xc8d0d9abfcc80cbd5973c5c378d1160a85ff1e591496a1d0f3b5d4d321279e24,43,0x125bf69c61af3c60ca1c0df5bbbce503cff7b1ae,109247,100500000000,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x0000000000000000000000000000000000000000000004d19fc85f3217800000...,[0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d82...,2020-11-02 00:00:58+00:00,11174331,DAI,Swap
4,0x121bbbd7404a26df4ee2f544c60e2cb198ed96d0b74fd9fd8038b686e41135a0,15,0xc1b9e80222f634246f0f38535e0a58203603108d,114126,35000000001,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x00000000000000000000000000000000000000000000014d22bf9ce18d2dbeb8...,[0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d82...,2020-11-02 00:01:19+00:00,11174332,DAI,Swap
5,0x121bbbd7404a26df4ee2f544c60e2cb198ed96d0b74fd9fd8038b686e41135a0,15,0xc1b9e80222f634246f0f38535e0a58203603108d,114126,35000000001,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x00000000000000000000000000000000000000000098d1ef6736c0db88ec5323...,[0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1],2020-11-02 00:01:19+00:00,11174332,DAI,Sync
6,0xc787a7cf0ec4a9ef094a24132eb8a61fee06a13301f2818bd3de64d7fa5734a8,34,0x29151005083afc6746dcecafb637974f248a3d74,114126,35000000001,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x00000000000000000000000000000000000000000098d4f4bbf34e274b90224d...,[0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1],2020-11-02 00:01:25+00:00,11174333,DAI,Sync
7,0xc787a7cf0ec4a9ef094a24132eb8a61fee06a13301f2818bd3de64d7fa5734a8,34,0x29151005083afc6746dcecafb637974f248a3d74,114126,35000000001,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x00000000000000000000000000000000000000000000030554bc8d4bc2a3cf2a...,[0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d82...,2020-11-02 00:01:25+00:00,11174333,DAI,Swap
8,0x106faf90667b9f9f396d281165856156b38c783e3ff40735a0c73e1c38b25c0f,17,0xce8224b761fcd0a839b2e9a091c498a6537ff35d,110298,26000000000,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x0000000000000000000000000000000000000000000000000000000000000000...,[0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d82...,2020-11-02 00:01:36+00:00,11174334,DAI,Swap
9,0x106faf90667b9f9f396d281165856156b38c783e3ff40735a0c73e1c38b25c0f,17,0xce8224b761fcd0a839b2e9a091c498a6537ff35d,110298,26000000000,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x00000000000000000000000000000000000000000098d4e6fb678d67aae4f090...,[0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1],2020-11-02 00:01:36+00:00,11174334,DAI,Sync


In [69]:
#uniswap = pd.read_pickle('tbu')
#uniswap.to_pickle('uniswap_events.pickle')

In [111]:
uniswap = uniswap[uniswap['event'] == 'Swap']

In [112]:
def getEthAmountInFromTopics(data):
    data = data[2:]
    parts = [data[i:i+64] for i in range(0, len(data), 64)]
    return int(parts[1],16)
    
def getDAIAmountOutFromTopics(data):
    data = data[2:]
    parts = [data[i:i+64] for i in range(0, len(data), 64)]
    return int(parts[2],16)

uniswap['eth_amount_in'] = uniswap['data'].apply(getEthAmountInFromTopics)
uniswap['dai_amount_out'] = uniswap['data'].apply(getDAIAmountOutFromTopics)

In [ ]:
uniswap = uniswap[(uniswap['eth_amount_in'] != 0) & (uniswap['dai_amount_out'] != 0)]
uniswap.drop_duplicates(subset='transactionHash', keep=False, inplace=True)

In [113]:
uniswap['transaction_fee'] = uniswap['gas_used'] * uniswap['gas_price']
uniswap['eth_used'] = uniswap['eth_amount_in'] * uniswap['transaction_fee']

In [116]:
uniswap['eth_price_dai'] = uniswap['dai_amount_out'] / uniswap['eth_amount_in']
uniswap['dai_price_eth'] = uniswap['eth_amount_in'] / uniswap['dai_amount_out']
uniswap

,transactionHash,transaction_index,transaction_sender,gas_used,gas_price,address,data,topics,block_timestamp,blockNumber_dec,contract,event,eth_amount_in,dai_amount_out,transaction_fee,eth_used,dai_price_eth,eth_price_dai
8,0x106faf90667b9f9f396d281165856156b38c783e3ff40735a0c73e1c38b25c0f,17,0xce8224b761fcd0a839b2e9a091c498a6537ff35d,110298,26000000000,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x0000000000000000000000000000000000000000000000000000000000000000...,[0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d82...,2020-11-02 00:01:36+00:00,11174334,DAI,Swap,650000000000000000,253682067964535976381,2867748000000000,1864036200000000000000000000000000,0.002562,390.280105
15,0xf5af1d562538ab7bf24d2356757b78055406a8a49cc512053f5cbd7ab2dc9b18,84,0xce8224b761fcd0a839b2e9a091c498a6537ff35d,176150,27000000000,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x0000000000000000000000000000000000000000000000000000000000000000...,[0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d82...,2020-11-02 00:02:09+00:00,11174336,DAI,Swap,792423932187323446,309470982455659659081,4756050000000000,3768807842679519675348300000000000,0.002561,390.537148
16,0x609af34848377b876a27bb77e5b32833eaa51dd4318a6bbf711202ddc41bbeb5,165,0xe04749d286f4190aefa28f5c607cdc4d08eac0a0,174891,26000000000,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x0000000000000000000000000000000000000000000000000000000000000000...,[0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d82...,2020-11-02 00:02:09+00:00,11174336,DAI,Swap,10590000000000000000,4135688906850244623266,4547166000000000,48154487940000000000000000000000000,0.002561,390.527753
36,0xb2f2b3cd54a89b8500f5bc6cf9ada4d8b091472f9cd3f97f587fed8c480a57f7,75,0x4e77efb547be0c37b17cb22eff384dfc240e13a6,976551,24100000000,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x0000000000000000000000000000000000000000000000000000000000000000...,[0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d82...,2020-11-02 00:03:33+00:00,11174343,DAI,Swap,115716670908768630,45275175770744727808,23534879100000000,2723377859692356836922633000000000,0.002556,391.258886
42,0x12d67ee7714c9b6d82986ea0c0e4763db760f6e5dfdb2e9bff6b165c14f84ab8,72,0x4d4d58e4dfefe4dafdff62d0945432c0c7425ebf,306191,25000000000,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x0000000000000000000000000000000000000000000000000000000000000000...,[0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d82...,2020-11-02 00:04:38+00:00,11174351,DAI,Swap,49700451850233432,19447310301687286833,7654775000000000,380445776311870619437800000000000,0.002556,391.290412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180022,0x35bd8361a2b8c4bbebdf15d87ca2fed4cae225c77cea351a2513184f23ba32bf,137,0x6455c07be90f40d51b041e128b2c4b00c56de508,125286,26400000000,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x0000000000000000000000000000000000000000000000000000000000000000...,[0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d82...,2020-11-20 23:57:17+00:00,11298064,DAI,Swap,1900000000000000000,958750349143459717010,3307550400000000,6284345760000000000000000000000000,0.001982,504.605447
180024,0xdaf9f656118078ee13d1e6678a71d9203d776bc502ca48da6c514a8c7d03f8c2,2,0x92048db9d572f3d153d415a41502ad20e9756904,194238,43810416667,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x0000000000000000000000000000000000000000000000000000000000000000...,[0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d82...,2020-11-20 23:57:28+00:00,11298065,DAI,Swap,710434732795108185,358482923296651465787,8509647712564746,6045549298856438904884828887046010,0.001982,504.596561
180026,0x382e9b350d9c84b034f20536dc725f972b6e78e5fe246d5e03c16c9c3f85e360,70,0xc371f9f3fa339d77d49831210ca6e71634f9ae5e,189389,41000000000,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x0000000000000000000000000000000000000000000000000000000000000000...,[0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d82...,2020-11-20 23:57:39+00:00,11298066,DAI,Swap,2

## Balancer Events

In [15]:
contracts = {
    '0x7afe74ae3c19f070c109a38c286684256adc656c'.lower(): 'ETH/DAI'
}

contracts

{'0x7afe74ae3c19f070c109a38c286684256adc656c': 'ETH/DAI'}

In [17]:
query = """
SELECT 
  events.transaction_hash AS transactionHash,
  events.transaction_index,
  txns.from_address AS transaction_sender, 
  txns.receipt_gas_used AS gas_used,
  txns.gas_price,
  events.address,
  events.data,
  events.topics,
  events.block_timestamp,
  events.block_number AS blockNumber_dec
FROM
  `bigquery-public-data.crypto_ethereum.logs` AS events
INNER JOIN
  `bigquery-public-data.crypto_ethereum.transactions` AS txns
ON
  events.transaction_hash = txns.hash
WHERE TRUE
  AND events.block_timestamp >= {from_block_ts} 
  AND events.block_timestamp <= {to_block_ts}
  AND txns.block_timestamp >= {from_block_ts} --might seem redundant, but because of partitioning this reduces cost
  AND txns.block_timestamp <= {to_block_ts} --might seem redundant, but because of partitioning this reduces cost
  AND txns.to_address IN ('0x3e66b66fd1d0b02fda6c811da9e0547970db2f21')
  AND events.address IN ({contract_list})
""".format(
    from_block_ts=FROM_BLOCK_TIMESTAMP, 
    to_block_ts=TO_BLOCK_TIMESTAMP, 
    contract_list=(','.join(["'{}'".format(k) for k in list(contracts.keys())])))

query_job = client.query(query)
iterator = query_job.result(timeout=MY_TIMEOUT)
rows = list(iterator)
balancer = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
balancer['contract'] = balancer['address'].apply(lambda x: contracts[x])

balancer.head(10)

,transactionHash,transaction_index,transaction_sender,address,data,topics,block_timestamp,blockNumber_dec,contract
0,0x7ba2ed40682c1f252734ecb919ec7748af28267955b1e46cef626c2a4e4fa800,49,0xe5a7819e296ccf03f0b211ab8f895466ee2e5e85,0x7afe74ae3c19f070c109a38c286684256adc656c,0x0000000000000000000000000000000000000000000000000000000000000020...,[0x8201aa3f0000000000000000000000000000000000000000000000000000000...,2020-11-18 17:28:57+00:00,11283273,ETH/DAI
1,0xc39cf280945a1c6d5637e7aa28097a769ac9309cff27c938975627bee82f8324,192,0x69e6717964273a4b031242bfcc904469ed7171dd,0x7afe74ae3c19f070c109a38c286684256adc656c,0x00000000000000000000000000000000000000000000001043561a8829300000...,[0x908fb5ee8f16c6bc9bc3690973819f32a4d4b10188134543c88706e0e1d4337...,2020-11-18 16:33:05+00:00,11283002,ETH/DAI
2,0x15b107396b75f418b2c3be6a5e5aa3c193f7dbfed1fced57123033eee93fdb0f,105,0x7185f2360f2498b0c4f695dce3775f2b0eb6fab6,0x7afe74ae3c19f070c109a38c286684256adc656c,0x000000000000000000000000000000000000000000000000156731e18419119d...,[0x908fb5ee8f16c6bc9bc3690973819f32a4d4b10188134543c88706e0e1d4337...,2020-11-18 12:42:01+00:00,11281988,ETH/DAI
3,0xb61733699493eb96a717671f67e8afdc1b2ccee04f4e584f2fe6dc4479083d33,74,0x9a728949a904a54b6eeecd21c9b67d48d00cdca8,0x7afe74ae3c19f070c109a38c286684256adc656c,0x0000000000000000000000000000000000000000000000000a217b21de090000...,[0x908fb5ee8f16c6bc9bc3690973819f32a4d4b10188134543c88706e0e1d4337...,2020-11-18 02:24:35+00:00,11279232,ETH/DAI
4,0xb61733699493eb96a717671f67e8afdc1b2ccee04f4e584f2fe6dc4479083d33,74,0x9a728949a904a54b6eeecd21c9b67d48d00cdca8,0x7afe74ae3c19f070c109a38c286684256adc656c,0x0000000000000000000000000000000000000000000000000000000000000020...,[0x8201aa3f0000000000000000000000000000000000000000000000000000000...,2020-11-18 02:24:35+00:00,11279232,ETH/DAI
5,0xc49817cb1b987d28a46bbdb8bf12789f58918dd274971731d5ba6f38e4645e9f,249,0x9dabf4ba0018be21c2a07b06affcdb8c77d9063f,0x7afe74ae3c19f070c109a38c286684256adc656c,0x0000000000000000000000000000000000000000000000000000000000000020...,[0x8201aa3f0000000000000000000000000000000000000000000000000000000...,2020-11-18 07:13:05+00:00,11280513,ETH/DAI
6,0xc39cf280945a1c6d5637e7aa28097a769ac9309cff27c938975627bee82f8324,192,0x69e6717964273a4b031242bfcc904469ed7171dd,0x7afe74ae3c19f070c109a38c286684256adc656c,0x0000000000000000000000000000000000000000000000000000000000000020...,[0x8201aa3f0000000000000000000000000000000000000000000000000000000...,2020-11-18 16:33:05+00:00,11283002,ETH/DAI
7,0x0c2b53a0f23194d66d9f85a2b048fe01bf5a53a199db868ee906187b7ea88c7a,39,0x6d4166eb4ecb2669691e22a057c50a75fb8ab21d,0x7afe74ae3c19f070c109a38c286684256adc656c,0x0000000000000000000000000000000000000000000000000000000000000020...,[0x8201aa3f0000000000000000000000000000000000000000000000000000000...,2020-11-18 01:07:22+00:00,11278859,ETH/DAI
8,0xa9ce7f50cf491946829df0cc2ef58166c57c8930125e498d4ca4f546ebac0f2c,172,0x6d4166eb4ecb2669691e22a057c50a75fb8ab21d,0x7afe74ae3c19f070c109a38c286684256adc656c,0x0000000000000000000000000000000000000000000000000000000000000020...,[0x8201aa3f0000000000000000000000000000000000000000000000000000000...,2020-11-18 00:55:25+00:00,11278810,ETH/DAI
9,0x7ba2ed40682c1f252734ecb919ec7748af28267955b1e46cef626c2a4e4fa800,49,0xe5a7819e296ccf03f0b211ab8f895466ee2e5e85,0x7afe74ae3c19f070c109a38c286684256adc656c,0x00000000000000000000000000000000000000000000000200f602c13ae342b7...,[0x908fb5ee8f16c6bc9bc3690973819f32a4d4b10188134543c88706e0e1d4337...,2020-11-18 17:28:57+00:00,11283273,ETH/DAI


In [85]:
sql = """
    select * from `blockchain-etl.ethereum_balancer.view_token_balances_subset`
    where address in ('0x7afe74ae3c19f070c109a38c286684256adc656c')
    and block_number >= 12403257 
    and block_number <= 12448607
    and balance > 0
"""

In [89]:
query_job = client.query(sql)
iterator = query_job.result(timeout=MY_TIMEOUT)
rows = list(iterator)
balances = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))

In [110]:
balances.head(10)

,token_address,address,block_number,balance
0,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,12404795,1.864131e+24
1,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,12404814,1.864131e+24
2,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,12404823,1.864131e+24
3,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,12404825,1.864131e+24
4,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,12404881,1.864131e+24
5,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,12404800,1.864131e+24
6,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,12404898,1.864131e+24
7,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,12404831,1.864131e+24
8,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,12404858,1.864131e+24
9,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,12405393,1.866496e+24
